In [1]:
import json
import shutil

import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import tiledb
import pdal
from pybabylonjs import Show as show

In [2]:
#!wget -nc "https://github.com/PDAL/data/blob/master/workshop/autzen.laz?raw=true" -O "autzen.laz"

In [3]:
#try:
#    shutil.rmtree("autzen")
#    pass
#except:
#    pass

In [4]:
pipeline1 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen1",chunk_size=10000000)
)

# count = pipeline1.execute()  

In [5]:
pipeline2 = (
  pdal.Reader("autzen.laz") | 
  pdal.Writer.tiledb(array_name="autzen2",x_tile_size=10, y_tile_size=10, z_tile_size=1,chunk_size=10000000)
)

# count = pipeline2.execute() 

In [6]:
pipeline3 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen3",chunk_size=10000000,data_tile_capacity=100000)
)

# count = pipeline3.execute()  

In [7]:
pipeline4 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen4",chunk_size=100000,data_tile_capacity=100000)
)

# count = pipeline4.execute()  

In [8]:
pipeline5 = (
  pdal.Reader("autzen.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen5",chunk_size=100000)
)

# count = pipeline5.execute()  

## Point cloud

In [9]:
%%time
with tiledb.open("autzen1") as arr:
    df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

df.head()

CPU times: user 10.1 s, sys: 704 ms, total: 10.8 s
Wall time: 1.16 s


,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime,Red,Green,Blue,X,Y,Z
0,1,1,1,1,0,2,-1.0,122,7331,248289.538944,111,112,98,637475.45,851804.30,415.49
1,1,1,1,1,0,2,10.0,122,7332,248680.116914,114,119,113,637174.99,852014.95,415.49
2,1,1,1,0,0,2,-12.0,124,7330,247564.723623,134,136,113,637371.69,851840.65,415.51
3,1,1,1,0,0,2,0.0,124,7331,248288.150659,138,145,124,637244.90,851873.25,415.51
4,1,1,1,1,0,2,-11.0,124,7330,247565.570015,112,114,108,637169.48,851998.29,415.51


In [10]:
data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'] / 255.0,
    'Green': df['Green'] / 255.0,
    'Blue': df['Blue'] / 255.0
}

In [11]:
show.from_dict(data=data,
                style = 'pointcloud',
                width = 800,
                height = 600,
                z_scale = .2,
                wheel_precision = 50)

BEFORE VALIDATE
{'style': 'pointcloud', 'width': 800, 'height': 600, 'z_scale': 0.2, 'wheel_precision': '50', 'extents': [636800.02, 637798.9700000001, 851000.03, 852998.97, 415.49, 598.15], 'time': None, 'data': {'X': 0          637475.45
1          637174.99
2          637371.69
3          637244.90
4          637169.48
             ...    
1309350    637640.48
1309351    637614.80
1309352    637295.63
1309353    637296.25
1309354    637296.25
Name: X, Length: 1309355, dtype: float64, 'Y': 0          851804.30
1          852014.95
2          851840.65
3          851873.25
4          851998.29
             ...    
1309350    852997.90
1309351    852997.76
1309352    852997.59
1309353    852998.12
1309354    852998.79
Name: Y, Length: 1309355, dtype: float64, 'Z': 0          415.49
1          415.49
2          415.51
3          415.51
4          415.51
            ...  
1309350    471.46
1309351    471.59
1309352    472.11
1309353    472.34
1309354    473.06
Name: Z, Length: 1309355, d

TraitError: '50' is not of type 'number'

Failed validating 'type' in schema['properties']['wheel_precision']:
    {'type': 'number'}

On instance['wheel_precision']:
    '50'

## MBRS

### Default sorting

In [ ]:
show.from_array(array='autzen1',
                style='mbrs',
                width=800,
                height=600,
                z_scale = 0.5,
                wheel_precision = 50)

### Optimized sorting - visual tuning - 2.5 x faster loading of data from array

In [ ]:
show.from_array(array='autzen5',
                style='mbrs',
                width=800,
                height=600,
                z_scale = 0.2)

In [ ]:
%%time
with tiledb.open("autzen1") as arr:
    df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

In [ ]:
%%time
with tiledb.open("autzen5") as arr:
    df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

## 4D 

In [ ]:
# %%time
# with tiledb.open("autzen5") as arr:
#     df = pd.DataFrame(arr[636800:637800, 851000:853000, 406.14:615.26])

# data = {
#     'X': df['X'],
#     'Y': df['Y'],
#     'Z': df['Z'],
#     'Red': df['Red'] / 255.0,
#     'Green': df['Green'] / 255.0,
#     'Blue': df['Blue'] / 255.0,
#     'GpsTime': df['GpsTime']}

In [ ]:
# show.from_dict(data=data,
#                       style = 'pointcloud',
#                         width = 800,
#                         height = 600,
#                         z_scale = .2,
#                         time = True)